#### Current issues: 
- Ending dataframe not ready (please use my current result to set up a dataframe structure)
- Current cost calculated ~ 300,000,000 (my termial gives me the result, but it will still be nice if Victor's excel can explain specifics)

In [71]:
# Import relevant libraries
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import pulp as pl
pd.set_option('display.max_rows', 200)

In [72]:
# Import datasets
carbon_emissions_df  = pd.read_csv('./data/carbon_emissions.csv')
resale_df = pd.read_csv('./data/cost_profiles.csv')
demand_df  = pd.read_csv('./data/demand.csv')
fuels_df = pd.read_csv('./data/fuels.csv')
vehicles_df  = pd.read_csv('./data/vehicles.csv')
vehicles_fuels_df  = pd.read_csv('./data/vehicles_fuels.csv')
sample_submission_df = pd.read_csv('./data/sample_submission.csv') 

### Edit vehicles_df to include all distance bucket selections

In [73]:
# Define a function to generate additional rows based on 'Distance'
def generate_rows(row):
    if row['Distance'] == 'D4':
        return pd.concat([
            pd.DataFrame([row]),
            pd.DataFrame([{**row, 'Distance': 'D3'}]),
            pd.DataFrame([{**row, 'Distance': 'D2'}]),
            pd.DataFrame([{**row, 'Distance': 'D1'}])
        ])
    elif row['Distance'] == 'D3':
        return pd.concat([
            pd.DataFrame([row]),
            pd.DataFrame([{**row, 'Distance': 'D2'}]),
            pd.DataFrame([{**row, 'Distance': 'D1'}])
        ])
    elif row['Distance'] == 'D2':
        return pd.concat([
            pd.DataFrame([row]),
            pd.DataFrame([{**row, 'Distance': 'D1'}])
        ])
    else:
        return pd.DataFrame([row])

# Apply the function to each row and concatenate the results
vehicles_df_add = pd.concat(vehicles_df.apply(generate_rows, axis=1).tolist(), ignore_index=True)

Before we calculate, we will merge the dataframe vehicles_df, vehicles_fuels_df, and fuels_df.

First, merge fuels_df with vehicles_df_add using column `Year`

In [74]:
merged_vehicles_df = pd.merge(vehicles_fuels_df, vehicles_df_add, on='ID', how='outer')

In [75]:
merged_fuels_total_df = pd.merge(merged_vehicles_df, fuels_df, on=['Fuel', 'Year'], how = 'outer')

In [76]:
merged_fuels_total_df.head()

,ID,Fuel,Consumption (unit_fuel/km),Vehicle,Size,Year,Cost ($),Yearly range (km),Distance,Emissions (CO2/unit_fuel),Cost ($/unit_fuel),Cost Uncertainty (±%)
0,Diesel_S1_2023,B20,0.223016,Diesel,S1,2023,85000,102000,D4,3.04858,1.220845,0
1,Diesel_S1_2023,B20,0.223016,Diesel,S1,2023,85000,102000,D3,3.04858,1.220845,0
2,Diesel_S1_2023,B20,0.223016,Diesel,S1,2023,85000,102000,D2,3.04858,1.220845,0
3,Diesel_S1_2023,B20,0.223016,Diesel,S1,2023,85000,102000,D1,3.04858,1.220845,0
4,Diesel_S2_2023,B20,0.223852,Diesel,S2,2023,104000,106000,D4,3.04858,1.220845,0


### Initialize the Pulp problem and define decision variables

Unfortunately, we will assume the consumption will be maximised for selected vehicle for now.

In [77]:
# Initialize the PuLP problem
prob = pl.LpProblem("Fleet_Transition_Optimization", pl.LpMinimize)

# Define Fuel type
fuel_type = merged_fuels_total_df['Fuel'].unique().tolist()

# Define Distance Bucket type
distance_bucket = merged_fuels_total_df['Distance'].unique().tolist()

# Define decision variables
buy_vars = pl.LpVariable.dicts("Buy", 
                               ((year, vehicle_id) for year in range(2023, 2039) for vehicle_id in vehicles_df['ID']),
                               lowBound=0, cat='Integer')

use_vars = pl.LpVariable.dicts("Use", 
                               ((year, vehicle_id, fuel, db) for year in range(2023, 2039) for vehicle_id in vehicles_df_add['ID']
                               for fuel in fuel_type for db in distance_bucket),
                               lowBound=0, cat='Integer')

sell_vars = pl.LpVariable.dicts("Sell", 
                                ((year, vehicle_id) for year in range(2023, 2039) for vehicle_id in vehicles_df['ID']),
                                lowBound=0, cat='Integer')

### Define objective function

The total cost consists of three components:
- Buying Cost for the vehicle
- Using Cost for the vehicle
- (Negative/Gain) Resale cost

Where the Using cost consists of three components:
- Fuel cost
- Insurance Cost
- Maintenance Cost

The Fuel cost is calculated by:

`D_v_f`: Distance travelled by vehicle `v` using fuel `f`. **(Assumed max)**
Unit: km

**multiplies by**

`N_v_f`: Number of vehicles of type `v` driving fuel type `f`.

**multiplies by**

`m_v_f`: Fuel consumption of vehicle of type `v` driving with fuel type `f`.
Unit: unit_fuel/km

**multiplies by**

`C_f_yr`: Cost of unit fuel of type `f` in the year `yr`.
Unit: $/unit_fuel

In [78]:
# First calculate buying costs
total_cost = pl.lpSum([
    # Buying cost
    buy_vars[(year, row['ID'])] * row['Cost ($)']
    for year in range(2023, 2039)
    for _, row in vehicles_df.iterrows()
])

In [79]:
# Add insurance and maintenance costs
for year in range(2023, 2039):
    for v in vehicles_df['ID']:
        purchase_year = int(v.split('_')[-1])
        if year >= purchase_year:
            age = year - purchase_year+1
            if age < 10:
                # Normal vehicle price
                purchase_cost = vehicles_df.loc[vehicles_df['ID'] == v, 'Cost ($)'].values[0]
                
                # Insurance percentage by age
                insurance_cost_pct = resale_df.loc[resale_df['End of Year'] == age, 'Insurance Cost %'].values[0] / 100
                
                # Maintenance percentage by age
                maintenance_cost_pct = resale_df.loc[resale_df['End of Year'] == age, 'Maintenance Cost %'].values[0] / 100
                
                # Insurance cost 
                insurance_cost = purchase_cost * insurance_cost_pct
                
                # Maintenance cost
                maintenance_cost = purchase_cost * maintenance_cost_pct
                
                # Add insurance cost to total cost
                total_cost += insurance_cost * pl.lpSum([use_vars[(year, v, fuel, db)] for db in distance_bucket for fuel in fuel_type])
                
                # Add maintenance cost to total cost
                total_cost += maintenance_cost * pl.lpSum([use_vars[(year, v, fuel, db)] for db in distance_bucket for fuel in fuel_type])

In [80]:
# Add fuel costs
for year in range(2023, 2039):
    for db in distance_bucket:
        for _, row in merged_fuels_total_df.iterrows():
            # Unique vehicle id
            vehicle_id = row['ID']

            # Max distance travelled by vehicle, unit: km
            distance = row['Yearly range (km)']

            # Fuel type
            fuel_t = row['Fuel']

            # Fuel cost, unit: $/unit
            fuel_cost = fuels_df[(fuels_df['Fuel'] == fuel_t) & (fuels_df['Year'] == year)]['Cost ($/unit_fuel)']

            # Fuel consumption, unit: unit/km
            fuel_consumption = row['Consumption (unit_fuel/km)']

            # Append calculated fuel cost to total cost
            ## Formula: Fuel_cost = number of vehicle v * Distance * Fule_Consumption * Fuel_cost
            total_cost += distance * fuel_cost * fuel_consumption * use_vars[(year, vehicle_id, fuel_t, db)]

In [81]:
merged_fuels_total_df

,ID,Fuel,Consumption (unit_fuel/km),Vehicle,Size,Year,Cost ($),Yearly range (km),Distance,Emissions (CO2/unit_fuel),Cost ($/unit_fuel),Cost Uncertainty (±%)
0,Diesel_S1_2023,B20,0.223016,Diesel,S1,2023,85000,102000,D4,3.048580,1.220845,0
1,Diesel_S1_2023,B20,0.223016,Diesel,S1,2023,85000,102000,D3,3.048580,1.220845,0
2,Diesel_S1_2023,B20,0.223016,Diesel,S1,2023,85000,102000,D2,3.048580,1.220845,0
3,Diesel_S1_2023,B20,0.223016,Diesel,S1,2023,85000,102000,D1,3.048580,1.220845,0
4,Diesel_S2_2023,B20,0.223852,Diesel,S2,2023,104000,106000,D4,3.048580,1.220845,0
...,...,...,...,...,...,...,...,...,...,...,...,...
1203,LNG_S3_2038,LNG,0.153945,LNG,S3,2038,235465,73000,D1,2.486188,0.926499,30
1204,LNG_S4_2038,LNG,0.154278,LNG,S4,2038,301395,118000,D4,2.486188,0.926499,30
1205,LNG_S4_2038,LNG,0.154278,LNG,S4,2038,301395,118000,D3,2.486188,0.926499,30
1206,LNG_S4_2038,LNG,0.154278,LNG,S4,2038,301395,118000,D2,2.486188,0.926499,30


#UNCERTAINTY
# Calculate the uncertain cost
total_uncertain_cost = pl.LpVariable('total_uncertain_cost', lowBound=0)

# Include cost uncertainty in total cost calculation
for year in range(2023, 2039):
    for vehicle_id in vehicles_df['ID']:
        fuel_cost = merged_fuels_total_df[(merged_fuels_total_df['ID'] == vehicle_id) & 
                                          (merged_fuels_total_df['Year'] == year)]['Cost ($/unit_fuel)'].values[0]
        uncertainty_pct = merged_fuels_total_df[(merged_fuels_total_df['ID'] == vehicle_id) & 
                                                (merged_fuels_total_df['Year'] == year)]['Cost Uncertainty (±%)'].values[0]
        uncertain_cost = fuel_cost * (1 + uncertainty_pct / 100)
        total_uncertain_cost += uncertain_cost * buy_vars[(year, vehicle_id)]

In [82]:
# check if consumption is correct

In [83]:
# Add (Negative/Gain) Resale cost
for year in range(2023, 2039):
    for vehicle_id in vehicles_df['ID']:
        purchase_year = int(vehicle_id.split('_')[-1])
        if year > purchase_year:
            age = year - purchase_year + 1
            if age < 10:
                purchase_cost = merged_fuels_total_df.loc[merged_fuels_total_df['ID'] == vehicle_id, 'Cost ($)'].values[0]
                resale_value_pct = resale_df.loc[resale_df['End of Year'] == age, 'Resale Value %'].values[0] / 100
                resale_value = purchase_cost * resale_value_pct
                total_cost -= resale_value * sell_vars[(year, vehicle_id)]

In [ ]:
#attempting to hard-code cost limitation and see what happens with output
prob += total_cost <=65000000

#TODO: cost uncertainty code
#marking down...out of time for additional cost uncertainty code

prob += total_cost + total_uncertain_cost <= 60000000

***

### Setting Constraints

In [85]:
# 1. Ensure vehicles can only be bought in the specific year
for year in range(2023, 2039):
    for v in merged_fuels_total_df['ID']:
        purchase_year = int(v.split('_')[-1])
        if year != purchase_year:
            prob += buy_vars[(year, v)] == 0

In [86]:
# 2. Ensure vehicle has a 10-year life and must be sold by the end of 10th year, and also must buy, then sell.
for v in merged_fuels_total_df['ID']:
    purchase_year = int(v.split('_')[-1])
    if purchase_year+10 <2040:
        prob += buy_vars[(purchase_year, v)] == pl.lpSum([sell_vars[(y, v)] for y in range(purchase_year, purchase_year+10)])
    else:
        prob += buy_vars[(purchase_year, v)] >= pl.lpSum([sell_vars[(y, v)] for y in range(purchase_year, 2039)])

In [87]:
# 3. Ensure vehicles are used only if they have been bought4
for v in merged_fuels_total_df['ID']:
    current = 0
    for y in range(2023, 2039):
        current += buy_vars[y, v]
        prob += current >= pl.lpSum([use_vars[(y, v, fuel, db)]  for fuel in fuel_type for db in distance_bucket])
        current -= sell_vars[y,v]

In [88]:
# 4. Every year at most 20% of the vehicles in the existing fleet can be sold
sell = pl.lpSum([sell_vars[2023, v] for v in vehicles_df['ID'] if int(v.split('_')[-1]) == 2023])
buy = pl.lpSum([buy_vars[2023, v] for v in vehicles_df['ID'] if int(v.split('_')[-1]) == 2023])
prob += sell<= 0.2 * buy

for year in range(2024, 2039):
    sell_temp = pl.lpSum([sell_vars[year, v] for v in vehicles_df['ID'] if int(v.split('_')[-1]) <= year])
    buy += pl.lpSum([buy_vars[year, v] for v in vehicles_df['ID'] if int(v.split('_')[-1]) == year])
    prob += sell_temp<= 0.2 * (buy-sell)
    sell += sell_temp

In [89]:
# 5. Ensure all vehicles can only use selected fuel type
for year in range(2023, 2039):
    for v in merged_fuels_total_df['ID']:
        vt = v.split('_')[0]
        for ft in fuel_type:
            for db in distance_bucket:
                if ((vt == 'BEV') & (ft != 'Electricity')) | ((vt == 'Diesel') & (ft not in ['B20', 'HVO'])) | ((vt == 'LNG') & (ft not in ['LNG', 'BioLNG'])):
                    prob += use_vars[(year, v, ft, db)] == 0  

In [90]:
# 6. Ensure vehicle can't be used if not belongs to demand bucket defined in original dataframe
for year in range(2023, 2039):
    for db in ['D1', 'D2', 'D3', 'D4']:
        for v in vehicles_df['ID']:
            if merged_fuels_total_df[(merged_fuels_total_df['ID'] == v)& (merged_fuels_total_df['Distance'] == db)].empty:
                prob += use_vars[(year, v, ft, db)] == 0    

In [91]:
# 7. Yearly demand must be met for each distance bucket
for year in range(2023, 2039):
    for db in ['D1', 'D2', 'D3', 'D4']:
        for size in ['S1', 'S2', 'S3', 'S4']:
            demand = demand_df[(demand_df['Year'] == year) & (demand_df['Distance'] == db) & (demand_df['Size'] == size)]['Demand (km)'].values[0]
            prob += pl.lpSum([use_vars[(year, v, fuel, db)] * vehicles_df_add.loc[vehicles_df_add['ID'] == v , 'Yearly range (km)'].values[0]
                           for v in vehicles_df_add[(vehicles_df_add['Distance'] == db) & (vehicles_df_add['Size'] == size)]['ID'] for fuel in fuel_type]) >= demand

In [92]:
# 8. Carbon emission limits must be respected
for year in range(2023, 2039):
    total_emissions = 0
    for _, row in merged_fuels_total_df.iterrows():
        # Unique vehicle id
        vehicle_id = row['ID']

        # Max distance travelled by vehicle, unit: km
        distance = row['Yearly range (km)']

        # Fuel type
        fuel_t = row['Fuel']
        
        # Carbon emissions, unit: (CO2/unit_fuel)
        emissions = row['Emissions (CO2/unit_fuel)']

        # Fuel consumption, unit: unit/km
        fuel_consumption = row['Consumption (unit_fuel/km)']

        # Calculate total emissions
        total_emissions += pl.lpSum([use_vars[(year, vehicle_id, fuel_t, row['Distance'])] * fuel_consumption * distance * emissions])
    prob += total_emissions <= carbon_emissions_df[carbon_emissions_df['Year'] == year]['Carbon emission CO2/kg'].values[0]

***

In [93]:
# Define solver parameters so that won't run infinitely
solver = pl.PULP_CBC_CMD(
    timeLimit=60,         # Maximum time in seconds
    gapRel=0.01,           # Relative gap tolerance       
    #threads=4,              # Number of threads to use
    msg=True                # Display solver messages
)

In [94]:
prob.solve(solver)

Welcome to the CBC MILP Solver 
Version: 2.10.3 
Build Date: Dec 15 2019 

command line - /Users/tonichan/anaconda3/envs/shell2024hackathon/lib/python3.12/site-packages/pulp/solverdir/cbc/osx/64/cbc /var/folders/j5/0__sh29d4jb7tpc11thjxmv00000gn/T/6f742488124349d0b93398aead2c88e3-pulp.mps -sec 60 -ratio 0.01 -timeMode elapsed -branch -printingOptions all -solution /var/folders/j5/0__sh29d4jb7tpc11thjxmv00000gn/T/6f742488124349d0b93398aead2c88e3-pulp.sol (default strategy 1)
At line 2 NAME          MODEL
At line 3 ROWS
At line 283814 COLUMNS
At line 1512900 RHS
At line 1796710 BOUNDS
At line 1864296 ENDATA
Problem MODEL has 283809 rows, 67585 columns and 1093916 elements
Coin0008I MODEL read with 0 errors
seconds was changed from 1e+100 to 60
ratioGap was changed from 0 to 0.01
Option for timeMode changed from cpu to elapsed
Problem is infeasible - 9.08 seconds
Option for printingOptions changed from normal to all
Total time (CPU seconds):       9.63   (Wallclock seconds):       9.83



-1

In [95]:
#check
# for year in range(2023, 2039):
#     total_emissions = 0
#     for _, row in merged_fuels_total_df.iterrows():
#         # Unique vehicle id
#         vehicle_id = row['ID']

#         # Max distance travelled by vehicle, unit: km
#         distance = row['Yearly range (km)']

#         # Fuel type
#         fuel_t = row['Fuel']
        
#         # Carbon emissions, unit: (CO2/unit_fuel)
#         emissions = row['Emissions (CO2/unit_fuel)']

#         Fuel consumption, unit: unit/km
#         fuel_consumption = row['Consumption (unit_fuel/km)']

#         # Calculate total emissions
#         total_emissions += np.sum([pl.value(use_vars[(year, vehicle_id, fuel_t, row['Distance'])]) * fuel_consumption * distance * emissions])
#     print(total_emissions <= carbon_emissions_df[carbon_emissions_df['Year'] == year]['Carbon emission CO2/kg'].values[0])

In [96]:
# Show buying variable results
for y in range(2023, 2039):
    for v in vehicles_df['ID']:
        if pl.value(buy_vars[(y, v)]) >0:
            print(y,v, pl.value(buy_vars[(y, v)]))

2023 BEV_S1_2023 33.983088
2023 BEV_S3_2023 32.724671
2023 LNG_S1_2023 32.274618
2023 LNG_S3_2023 47.046123
2026 LNG_S1_2026 5.5226765
2028 BEV_S1_2025 9.4915392
2028 BEV_S1_2028 71.711686
2028 LNG_S3_2031 16.665329
2029 LNG_S4_2024 0.1970678
2029 LNG_S3_2028 77.7082
2030 BEV_S1_2024 12.297627
2031 BEV_S1_2031 46.654284
2031 BEV_S3_2031 79.385767
2032 Diesel_S3_2032 18.092712
2032 LNG_S1_2033 15.810402
2033 Diesel_S1_2035 76.293814
2033 LNG_S1_2033 3214176.6
2034 BEV_S1_2034 4.3191863
2035 BEV_S3_2035 83.059849
2036 BEV_S3_2036 14.879156


In [97]:
# Show using variable results
for y in range(2023, 2039):
    for v in vehicles_df['ID']:
        for d in distance_bucket:
            for f in fuel_type:
                if pl.value(use_vars[(y, v, f, d)]) >0:
                    print((y,v,f, d, pl.value(use_vars[(y, v, f, d)]) ))

(2023, 'BEV_S1_2023', 'Electricity', 'D1', 33.983088)
(2023, 'BEV_S1_2028', 'Electricity', 'D2', 25.461706)
(2023, 'BEV_S2_2028', 'B20', 'D2', 13.049019)
(2023, 'BEV_S2_2030', 'B20', 'D3', 7.3396981)
(2023, 'BEV_S4_2023', 'Electricity', 'D1', 7.5423814)
(2023, 'BEV_S4_2031', 'B20', 'D3', 1.0076186)
(2023, 'BEV_S4_2033', 'B20', 'D2', 6.4112373)
(2023, 'BEV_S3_2023', 'Electricity', 'D1', 32.724671)
(2023, 'LNG_S1_2023', 'LNG', 'D3', 32.274618)
(2023, 'LNG_S2_2023', 'LNG', 'D1', 29.782057)
(2023, 'LNG_S4_2029', 'B20', 'D4', 0.015330508)
(2023, 'LNG_S3_2023', 'LNG', 'D3', 13.732411)
(2023, 'LNG_S3_2023', 'LNG', 'D2', 33.313712)
(2023, 'LNG_S3_2029', 'Electricity', 'D4', 2.8140548)
(2024, 'BEV_S1_2023', 'Electricity', 'D1', 33.983088)
(2024, 'BEV_S1_2028', 'BioLNG', 'D2', 26.629363)
(2024, 'BEV_S1_2036', 'Electricity', 'D3', 32.711804)
(2024, 'BEV_S1_2036', 'Electricity', 'D1', 1.2466863)
(2024, 'BEV_S2_2024', 'Electricity', 'D1', 31.848906)
(2024, 'BEV_S2_2033', 'BioLNG', 'D4', 1.261434)
(

In [98]:
# Show selling variable results
for y in range(2023, 2039):
    for v in vehicles_df['ID']:
        if pl.value(sell_vars[(y, v)]) is not None:
            if pl.value(sell_vars[(y, v)]) >0:
                print(y, v, pl.value(sell_vars[(y, v)]))

2024 BEV_S3_2023 1004486.2
2025 BEV_S1_2023 36.411078
2026 BEV_S2_2025 1004443.8
2027 LNG_S3_2023 1808058.0
2028 Diesel_S3_2023 1446440.1
2029 Diesel_S1_2028 4220734.6
2029 LNG_S3_2028 77.7082
2030 BEV_S3_2028 4632207.3
2031 BEV_S1_2029 5.1148137
2031 BEV_S2_2029 5887752.2
2031 Diesel_S2_2030 13.171726
2031 LNG_S1_2023 1960719.4
2032 BEV_S1_2028 33.636608
2032 BEV_S4_2031 19.468825
2032 BEV_S3_2029 4632135.3
2032 Diesel_S3_2027 3214229.0
2033 BEV_S1_2028 38.075078
2033 LNG_S2_2031 4017802.8
2034 BEV_S3_2031 79.385767
2034 Diesel_S2_2032 4017796.2
2035 BEV_S1_2034 4.3191863
2035 Diesel_S3_2027 803622.5
2035 LNG_S1_2034 3214193.5
2036 BEV_S3_2035 54.727388
2036 Diesel_S3_2032 14.080986
2036 LNG_S4_2028 7232039.2
2037 BEV_S3_2035 28.332462
2038 BEV_S3_2036 14.879156


In [99]:
output = []
for year in range(2023, 2039):
    for vehicle_id in vehicles_df['ID']:
        if pl.value(buy_vars[(year, vehicle_id)]) > 0:
            output.append({
                'Year':year,
                'ID': vehicle_id,
                'Num_Vehicles': int(pl.value(buy_vars[(year, vehicle_id)])),
                'Type': 'Buy',
                'Fuel': np.nan,
                'Distance_bucket':'NaN',
                'Distance_per_vehicle(km)': 0.0
            })
        for fuel in fuel_type:
            for db in distance_bucket:
                if pl.value(use_vars[(year, vehicle_id, fuel, db)]) > 0:
                    vehicle_info = merged_fuels_total_df[merged_fuels_total_df['ID'] == vehicle_id].iloc[0]
                    output.append({
                        'Year':year,
                        'ID': vehicle_id,
                        'Num_Vehicles': int(pl.value(use_vars[(year, vehicle_id, fuel, db)])),
                        'Type': 'Use',
                        'Fuel': fuel,
                        'Distance_bucket': db,
                        'Distance_per_vehicle(km)': vehicle_info['Yearly range (km)']
            })                                   
                    
        if pl.value(sell_vars[(year, vehicle_id)]) > 0:
            output.append({
                'Year':year,
                'ID': vehicle_id,
                'Num_Vehicles': int(pl.value(sell_vars[(year, vehicle_id)])),
                'Type': 'Sell',
                'Fuel': np.nan,
                'Distance_bucket':'NaN',
                'Distance_per_vehicle(km)': 0.0
            })

output_df = pd.DataFrame(output, columns=[
    'Year', 'ID', 'Num_Vehicles', 'Type', 'Fuel', 'Distance_bucket', 'Distance_per_vehicle(km)'
])

output_df.head()

,Year,ID,Num_Vehicles,Type,Fuel,Distance_bucket,Distance_per_vehicle(km)
0,2023,BEV_S1_2023,33,Buy,NaN,NaN,0.0
1,2023,BEV_S1_2023,33,Use,Electricity,D1,102000.0
2,2023,BEV_S1_2028,25,Use,Electricity,D2,102000.0
3,2023,BEV_S2_2028,13,Use,B20,D2,106000.0
4,2023,BEV_S2_2030,7,Use,B20,D3,106000.0


In [100]:
from datetime import datetime
output_df.to_csv(f"shell_hackathon_2024_output_{datetime.now()}.csv", index=False)